## Kensho Technologies Workshop - January 29th, 2019
### Linking Datasets

In [10]:
import numpy as np
import pandas as pd
import nltk

In [2]:
ishares = pd.read_csv('../data/ai_meetup_data/ishares_companies.csv')
wikidata = pd.read_csv('../data/ai_meetup_data/wikidata_companies.csv')
labels = pd.read_csv('../data/ai_meetup_data/labeled_data.csv')

In [3]:
ishares.head()

,ishares_id,Name,Country,Sector,Ticker,Market Value
0,0,ENEL,Italy,Utilities,ENEL,"35,606,728.17"
1,1,ENI,Italy,Energy,ENI,"30,934,624.17"
2,2,INTESA SANPAOLO,Italy,Financials,ISP,"25,171,216.82"
3,3,UNICREDIT,Italy,Financials,UCG,"17,460,161.24"
4,4,ASSICURAZIONI GENERALI,Italy,Financials,G,"11,332,119.83"


In [4]:
wikidata.head()

,wikidata_id,item_title,country
0,Q1191932,0verflow,NaN
1,Q4545742,1-2-3,Norway
2,Q4545758,1-800 Contacts,United States of America
3,Q4545749,1-800-FREE-411,United States of America
4,Q4545754,1-800-GOT-JUNK?,NaN


In [5]:
labels.head()

,ishares_id,wikidata_id
0,0,Q651222
1,1,Q565594
2,2,Q1343118
3,3,Q45568
4,4,Q670482


### To Dos:

1. What are differences in how names are written across datasets?
2. Find 100 new links between the datasets
3. Find 100 new entities in Dataset 1 not in Dataset 2

In [11]:
# rename columns of wikidata dataset to match relevant columns of ishares data
wikidata.rename(columns={'item_title': 'Name', 'country': 'Country'}, inplace=True)
wikidata.head()

,wikidata_id,Name,Country
0,Q1191932,0verflow,NaN
1,Q4545742,1-2-3,Norway
2,Q4545758,1-800 Contacts,United States of America
3,Q4545749,1-800-FREE-411,United States of America
4,Q4545754,1-800-GOT-JUNK?,NaN


## Exploratory Data Analysis

### 1. Metadata
What can I learn about each dataset from the information given, before getting into any NLP or feature engineering?

    -Country data
    -Industry data
    -Company Size
    -Alphabetical Distribution

In [12]:
# ishares data
ishares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3836 entries, 0 to 3835
Data columns (total 6 columns):
ishares_id      3836 non-null int64
Name            3836 non-null object
Country         3836 non-null object
Sector          3836 non-null object
Ticker          3836 non-null object
Market Value    3836 non-null object
dtypes: int64(1), object(5)
memory usage: 179.9+ KB


In [13]:
ishares.head()

,ishares_id,Name,Country,Sector,Ticker,Market Value
0,0,ENEL,Italy,Utilities,ENEL,"35,606,728.17"
1,1,ENI,Italy,Energy,ENI,"30,934,624.17"
2,2,INTESA SANPAOLO,Italy,Financials,ISP,"25,171,216.82"
3,3,UNICREDIT,Italy,Financials,UCG,"17,460,161.24"
4,4,ASSICURAZIONI GENERALI,Italy,Financials,G,"11,332,119.83"


In [20]:
countries = ishares.Country.unique()

In [19]:
for c in countries:
    

3836

### Notes

Ideas of how to break down into smaller categories:

    -by country
    -by first letter
    -by number of words
    -by wordmatch